### Programming Assignment 4.1: Salary Forecast Based on Job Description

- using linear regression, we will predict the salary based on the text description of the vacancy (and location and type of contract)

- используя линейную регрессию, спрогнозируем зарплату по текстовому описанию вакансии (и локации и типу контракта)

In [346]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl

from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import hstack

from sklearn.decomposition import PCA

In [289]:
df = pd.read_csv('salary-train.csv')
df.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


#### Data cleaning

In [203]:
# text to lowercase
df['FullDescription'] = df['FullDescription'].str.lower() 

# remove all except alphabetic symbols
df['FullDescription'] = df['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

# fillna
df['LocationNormalized'].fillna('nan', inplace=True)
df['ContractTime'].fillna('nan', inplace=True)

Transform texts to TF-IDF

In [245]:
texts_vectorizer = TfidfVectorizer(min_df=5)
texts_vect = texts_vectorizer.fit_transform(df['FullDescription'])

LocationNormalized и ContractTime являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют категориальными или номинальными. Типичный подход к их обработке — кодирование категориального признака с m возможными значениями с помощью m бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют one-hot-кодированием. 

**Лайфхак:**
если например город встречается один раз во всей выборке,то у нас будет бинарный признак, который принимает значение "1" только один раз – это не принсет нам никакой пользы, алгоритм (скорее всего) не сможет найти здесь зависимости. 

**РЕШЕНИЕ:** можнно объединить такие города в "один", назвать этот этот бинарный признак "редкие"

In [207]:
enc = DictVectorizer()
locations_and_contracts = enc.fit_transform(df[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [236]:
# X
X = hstack([texts_vect, locations_and_contracts])

# y
salary = df.values[:, -1]

#### Fit model – Ridge linear regression

In [242]:
model = Ridge(alpha=1, random_state=241)
model.fit(X, salary)

Ridge(alpha=1, random_state=241)

#### Prepare test data

In [290]:
tests = pd.read_csv('salary-test-mini.csv')
tests.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [291]:
tests['FullDescription'] = tests['FullDescription'].str.lower() 
tests['FullDescription'] = tests['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

tests_texts = texts_vectorizer.transform(tests['FullDescription'])
locs_and_contracts = enc.transform(tests[['LocationNormalized', 'ContractTime']].to_dict('records'))

X_test = hstack([tests_texts, locs_and_contracts])

#### Predict salary

In [292]:
model.predict(X_test)

array([56581.31152328, 37133.54332311])

### Programming Assignment 4.2: Stock Index Compilation (Составление фондового индекса)

- work with principal component analysis (метод главных компонент)
- use it to calculate the improved Dow Jones index

Метод главных компонент (principal component analysis, PCA) — это один из методов обучения без учителя, который позволяет сформировать новые признаки, являющиеся линейными комбинациями старых. При этом новые признаки строятся так, чтобы сохранить как можно больше дисперсии в данных. Иными словами, метод главных компонент понижает размерность данных оптимальным с точки зрения сохранения дисперсии способом.

In [317]:
close_prices_df = pd.read_csv('close_prices.csv')
close_prices_df.head()

,date,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,2013-09-23,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,2013-09-24,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,2013-09-25,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,2013-09-26,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,2013-09-27,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


In [361]:
djia_df = pd.read_csv('djia_index.csv')
djia_df.head()

,date,^DJI
0,2013-09-23,15401.379883
1,2013-09-24,15334.589844
2,2013-09-25,15273.259766
3,2013-09-26,15328.299805
4,2013-09-27,15258.240234


In [354]:
pca = PCA(n_components=10)
pca.fit(X)

pca.explained_variance_ratio_

array([0.73897118, 0.11007169, 0.04995088, 0.0287492 , 0.02215448,
       0.01931577, 0.00674853, 0.00614091, 0.00320594, 0.00305611])

In [355]:
count = 0
sum = 0
for i in range(len(pca.explained_variance_ratio_)):
    count+=1
    value = pca.explained_variance_ratio_[i]
    sum+= value
    if sum > 0.9:
        break
print("Need components %d" % count)

Need components 4


#### Коэффициент корреляции Пирсона между значением по первой компоненте и индексом Доу-Джонса

In [375]:
np.corrcoef(pca.transform(X)[:,0], djia_df['^DJI'])

array([[1.        , 0.90965222],
       [0.90965222, 1.        ]])

#### Какая компания имеет наибольший вес в первой компоненте?

In [400]:
idx = np.argmax(pca.components_[0])
pca.components_[0][idx]

0.5796839457473607

In [401]:
X.columns[idx]

'V'

### V stands for Visa